In [1]:
import requests
import csv
import os
from bs4 import BeautifulSoup
import pandas as pd
import re
import webbrowser
from tkinter import *

filmRef = {}

In [2]:
def get_pages(link,nb_pages):
    # link correspond au lien du site et nb_pages le nombre de page que l'on veut traiter (en sachant que 1 page = 50 films) donc pour chaque tour de boucle on ajoute dans la variable le lien du site avec 1 50 100 etc (variable page dans la boucle)
    
    tab_pages = []
    x=0
    page=1
    while(x < nb_pages):
        tmp = link + str(page)
        tab_pages.append(tmp)
        x=x+1
        page=page+50
    return tab_pages

In [3]:
def affichage(dict):
    # pas important juste pour afficher le dictionnaire (regarde sur internet documente toi sur les dictonnaires en python)
    for key1,dict2 in dict.items():
        print(key1)
        for keys,values in dict2.items():
            print(keys)
            print(values)

In [37]:
def ajoute_film(dict,num):
    #dict correspond au dictionnaire qui contient tout les films par exemple pour chaque nouveau films on aura dans le disctionnaire 
    # dict=( (1,("dictionnaire des données du film 1")) ; (2,("dictionnaire des données du film 2")) etc ) (le numero c'est la clé et le dictionnaire la valeur correspondante à la clé)
    # la fonction elle a en argument le dict dictionnaire principal et num le numero du film qu'on va ajouter

    numero=str(num)
    donnees_du_film={}
    #update ajoute à la suite dans le dictionnaire un numero et un dictionnaire vide qui va contenir les données du film
    dict.update({numero:donnees_du_film})   

In [4]:
def ajoute_donnees_film(dict,key,value,num):
    #Cette fonction va ajouter dans le dictionnaire vide les données associé au film 
    # On a en argument key et value qui peuvent representer par exemple key = Genre et genre = Horreur par exemple si on prend num = 1 (numero du film ou on veut ajouter les données)
    # on aura dans le dictionnaire principale dict = ( (1:("Genre":Horreur))) -> equivalent a dict = ( (num : ("key":value)))
    
    for key1,dict2 in dict.items():
        if key1 == num:
            #On parcours le dictionnaire principal dict qui sera de la forme dict=( (1,("dictionnaire des données du film 1")) ; (2,("dictionnaire des données du film 2"))
            # les numero des film ici correspondent a key1 et le numero du film ou on veut ajouter les données avec la fonction correspond a num
            # donc si key1 = num -> equivaut à dire, si le numero du film dans le dictionnaire est egal au numero du film où on veut ajouter les données
            # donc par exemple si on a un dictionnaire avec 2 films dedan de la forme dict=( (1,("dictionnaire des données du film 1")) ; (2,("dictionnaire des données du film 2")) et num = 2, on va dabord regarder le 1er film qui a une valeur de key1=1 donc puisque keys different de num on continue au suivant et sa marchera puisque num =2 et key1=2
            dict2.update({key:value})

In [5]:
def extraction(data,num_page):
    
    link = 'https://www.imdb.com/search/title/?release_date=2007-01-01,2020-07-01&sort=num_votes,desc&title_type=feature&ref_=adv_nxt&start='
    page = 1
    compteur_film=0
    compteur_global_film=0
    pages = get_pages(link,num_page) #Contient toutes les pages 

    #print(pages)
    position_film=0

    #pour chaque page, on extrait les donnée

    for code in pages:

        #permet de recuperer le code HTML de la page
        page = (requests.get(code)).content
        soup = BeautifulSoup(page,features="xml")
        
        with open("page_html.csv", "w", encoding="utf-8") as csv_file:
            writer = csv.writer(csv_file)
            writer.writerow(soup)

        soup.text
    
        film = {}

        #TITRE DU FILM

        #find permet de chercher dans le code HTML où se trouve l'information (la donnée que l'on cherche ), find_all permet de recuperer toute les données de la page puisque chaque données des films sont construite exactement pareils dans une page HTML
        # par exemple pour le titre du film, on remarque en regardant dans le code que la donnée se trouve dans la balise h3 ayant class=lister-item-header
        h3 = soup.find_all("h3", {"class": "lister-item-header"})
        #print(h3)

        for x in h3:
            for y in x.find_all('a', href=True, text=True):
                 # dans la balise h3, le titre se trouve dans la ligne de la balise a, on entre donc dans cette ligne avec find
                compteur_global_film=compteur_global_film+1
                ajoute_film(data,compteur_global_film)
                for i in y:   
                    #i correspond au titre, on l'ajoute avec la fonction permettant d'ajouter une donnée a un film
                    ajoute_donnees_film(data,'Titre',i,str(compteur_global_film))
                    
        
        #ensuite c'est pareil pour chaque donnée, on cherche où la donnée se trouve dans le code HTML, puis on entre dans les balises correspondante
        #Ref
        
        compteur_film=0+position_film*50

        h3 = soup.find_all("div", {"class": "ribbonize"})
        
        for elem in h3:
            compteur_film=compteur_film+1
            ajoute_donnees_film(data,'Ref',elem.get('data-tconst'),str(compteur_film))
        
        #Genre
        
        compteur_film=0+position_film*50
        h3 = soup.find_all("p", {"class": "text-muted"})
        
        for x in h3:
            for y in x.find_all("span", {"class": "genre"}):
                compteur_film=compteur_film+1
                for i in y:
                    genre=i.strip()
                    ajoute_donnees_film(data,'Genre',genre,str(compteur_film))
            

        #Duree
        
        compteur_film=0+position_film*50
        h3 = soup.find_all("p", {"class": "text-muted"})
        
        for x in h3:
            for y in x.find_all("span", {"class": "runtime"}):
                compteur_film=compteur_film+1
                for i in y:
                    ajoute_donnees_film(data,'Duree',i,str(compteur_film))
                    
        #Annee de Sortie
        
        compteur_film=0+position_film*50
        h3 = soup.find_all("h3", {"class":  "lister-item-header"})
        
        for x in h3:
            for y in x.find_all("span", {"class": "lister-item-year text-muted unbold"}):
                compteur_film=compteur_film+1
                for annee in y:
                    annee=annee.replace('(','')
                    annee=annee.replace(')','')
                    ajoute_donnees_film(data,'Annee',annee,str(compteur_film))
    
            
        #Nombre de vote
        
        compteur_film=0+position_film*50
        h3 = soup.find_all("meta", {"itemprop":  "ratingCount"})
        
        for i in h3:
            compteur_film=compteur_film+1
            ajoute_donnees_film(data,'Nombre de votes',i.get('content'),str(compteur_film))

        #Note du public
        
        compteur_film=0+position_film*50
        h3 = soup.find_all("meta", {"itemprop":  "ratingValue"})
        
        for z in h3:
            compteur_film=compteur_film+1
            ajoute_donnees_film(data,'Note du public',z.get('content'),str(compteur_film))
            
        #Note Critique
        
        compteur_film=0+position_film*50
        h3 = soup.find_all("span", {"class": "metascore"})    
        for z in h3:
            for i in z:
                compteur_film=compteur_film+1
                note_c=i.strip()
                ajoute_donnees_film(data,'Note Critique',note_c,str(compteur_film))
        
        #Recette du film  
        
        compteur_film=0+position_film*50
        h3 = soup.find_all("p", {"class": "sort-num_votes-visible"})
        for x in h3:
            for y in x.find_all("span", {"class": "ghost"}):
                compteur_film=compteur_film+1
                recette=y.next_element.next_element.next_element.next_element.next_element.next_element
                recette= recette.get('data-value')
                ajoute_donnees_film(data,'Recette',recette.replace('"', ''),str(compteur_film))   
    
                
        position_film=position_film+1

In [6]:
cond1 = False
cond2 = False


tabresult=[]
num = 1
cond1 = False
cond2 = False

In [7]:
def listeFilm(dict):
    # Cette fonction va servir a recuperer la liste des titres des films dans la base de donnéees -> les titres sont recuperer dans le tableau listfilm
    listfilm=[]
    for key1,dict2 in dict.items():
        for key2,value in dict2.items():
            if key2 == "Titre":
                listfilm.append(value)
    #on parcours le dictionnaire principal puis le dictionnaire contenant les données
    #le dictionnaire contentnt les données pour un film est de la forme ("Titre":"Iron man"),("Genre":"Action"),etc...
    # Si la clé est egal a "Titre", on prend la valeur associer a cette clé et on l'ajoute dans le tableau resultat
    return listfilm

In [9]:
def getTitreRef(dict,titre):

    # Cette fonction va juste servire a chercher la reference d'un film
    #On donne en argument le Titre du film où on veut recuperer la reference associé
    num = getNum(dict,titre)
    for key1,dict2 in dict.items():
        if key1 == num:
            for key2,value in dict2.items():
                if key2 == "Ref":
                    return value
    # Grace a la fonction getNum (fonction 16), on recupere le numero du titre du film, on entre dans le disctionnaire principal (on verifie si la clé est bien égal au film souhaité pour recuperer sa reference et pas celle d'un autre) puis on parcours le dictionnaire secondaire. Puis comme precedement, Si la clé est egal a "Ref", on prend la valeur associer a cette clé et on la retourne
    return None

In [10]:
def affichage(tab):
    for i in tab:
        print(i)

In [11]:
def getNum(dict,titre):
    # Pour un titre de film, on recupere son numero associé
    for key1,dict2 in dict.items():
        for key2,value in dict2.items():
            if key2 == "Titre":
                if value == titre:
                    return key1
    return None

In [12]:
def filtre(dict,num,valeur,critere):
    # num correspond au film que l'on veut tester
    #valeur correspond a la valeur que l'utilisateur aura entrer dans le programme de recommendation
    #Cette fonction va tester si le film (correspondant a num) possede bien le critere quon lui a donner
    # Par exemple, quand l'utilisateur va choisir un genre parmis la liste, la fonction va parcours tout les films de la base de données dict et tester si chacun d'eux possede bien le genre que l'utilisateur a entrer (de meme pour les années)
    
    for key1,dict2 in dict.items():
        if key1 == num:
            for key2,value in dict2.items():
                if key2 == critere:

                    l= value.split(',')
                    for i in l:
                        if i.replace(' ','') == valeur:
                            return True
    return False

In [13]:
def filtreGenre(dict,liste,genre):
    #pour chaque film de la liste, par exemple pour iron man avec le genre action, si l'utilisateur a entrer action, la fonction filtre (precedente fonction) va tester si il a bien le genre que l'utilisateur a entrer. Si c'est le cas, on ajoute le film dans la liste resultat
    listeRes=[]
        
    for film in liste:
        if filtre(dict,getNum(dict,film),genre,"Genre"):
            listeRes.append(film)
    return listeRes

In [14]:
def filtreAnnee(dict,liste,annee):
    #pareil que precedent mais pour les années
    listeRes=[]
        
    for film in liste:
            
        if filtre(dict,getNum(dict,film),annee,"Annee"):
            listeRes.append(film)
    return listeRes

In [15]:
boole=True

data={}
                                                                           
while True:
    
    print("-------------------------------------------------------")
    print("PROJECT : DATA SCIENCE APPLIED TO MOVIES")
    print("-------------------------------------------------------")
    
    print(" ------ 1:START  ;  2:EXIT  ----- ")

    var = input()

    #input est une fonction importante pour le programme de recommendation qui va servir à demander à l'utilisateur d'ecrire quelque chose
    # ce quelque chose sera recuperer dans la variable var

    if ( int(var) == 1):
        #si on decide de commencer le programme var sera egal à 1 donc on entre dans cette condition
        

        ###### on demande le nombre de film qu'on veut importer #####################
        print("How many movies do you want to import ? :")
        choix = [50,250,500,750,1000]
        cpt=0
        for i in choix:
            print(str(cpt+1)+" : " +str(i)+" movies", end = '   ')
            cpt+=1
        print()
        
        tmp = input("Select an answer : ")

        ###### on recupere la valeur demander à l'utilisateur dans tmp ####################

        # on parcours le tableau choix contenant les choix que l'utilisateur devait choisir precedement
        # le cpt correspond à la place du choix dans le tableau choix = [50,250,500,750,1000]
        # par exemple 500 sera à l'emplacement 3, si le choisir aura choisi 3 comme proposition, on utilise arrete la boucle for suivante
        # on parcours le tableau choix, chaque valeur dans le tableau va correspondre a i 

        cpt=1
        for i in choix :
            if cpt == int(tmp):
                nb = i/50
                # on divise par 50 pour obtenir le nombre de film que l'on voudra extraire
            cpt+=1
        
        extraction(data,nb)
        # on utilise la fonction extraction pour extraire les film du site
        
        df = pd.DataFrame.from_dict(data, orient="index")
        df.to_csv("data.csv")

        # Proposition des genres et des années de la meme maniere que precedement

        print("Select 1 Genre among the following proposals : ")
        print("Action - Adventure - Drama - Crime - Mystery - Sci-Fi - Biography - Fantasy - Thriller - Family - Romance - Comedy")
        Genre = input('Genre : ')
    
    ############### cette partie sert a tester si les valeur que l'utilisateur va mettre sont correct################
        if(Genre == "Action" or Genre == "Adventure" or Genre == "Drama" or Genre == "Crime" or Genre == "Mystery" or Genre == "Sci-Fi" or Genre == "Biography" or Genre == "Fantasy" or Genre =="Thriller" or Genre =="Family" or Genre == "Romance" or Genre == "Comedy" and Genre != None):
            cond1 = True
            #Si la valeur entrer par l'utilisateur est correct, la condition 1 est respecter 
            
        try:
            # On propose dentrer les dates 
            print("-------------------------------------------------------")
            print("Select a year between 2008 and 2020 : ")

            Annee = int(input("Year : "))
            
            #Si la valeur entrer par l'utilisateur est correct, la condition 2 est respecter
            if ( Annee >=2000 and Annee <= 2020 ):
                cond2=True

            # Si les 2 conditions sont True, on sort de cette boucle
            if ( cond1 and cond2 ):
                break
            
            if ( cond1 == False and cond2 == True):
                print("rror : Select a correct genre")
                
            if ( cond1 == True and cond2 == False):
                print("rror : Select a correct year")
                
        except ValueError:
            
            if ( cond1 == False):
                print("Error : Select a correct genre")
                
            print("Bad argument error")
            
        ##########################################################
    
    if ( int(var) == 2 ):
        boole=False
        break

 
if( boole):
    # IMPORTANT : la liste suivante va contenir les films filtrer selon les choix de l'utilisateur
    liste = filtreGenre(data,listeFilm(data),Genre)
    liste = filtreAnnee(data,liste,str(Annee))


################# Cette partie va permettrent a l'utilisateur de choisir un film dans la liste resultat et d'ouvrir une page sur le site IMDB concernant ce film  ###########################
while boole : 
    
    cpt=1
    print("Films :")
    for i in liste: 
        print(str(cpt)+" : "+i)
        cpt+=1
    
    num = input("Select your favorite movie: ")
    
    link='https://www.imdb.com/title/'
    cpt=1
    for i in liste:
        if (int(num) == cpt):
            ref = getTitreRef(data,i)
            webbrowser.open(link+ref)
            break
        cpt+=1
    break
########################################################

-------------------------------------------------------
PROJECT : DATA SCIENCE APPLIED TO MOVIES
-------------------------------------------------------
 ------ 1:START  ;  2:EXIT  ----- 
How many movies do you want to import ? :
1 : 50 movies   2 : 250 movies   3 : 500 movies   4 : 750 movies   5 : 1000 movies   
Select 1 Genre among the following proposals : 
Action - Adventure - Drama - Crime - Mystery - Sci-Fi - Biography - Fantasy - Thriller - Family - Romance - Comedy
-------------------------------------------------------
Select a year between 2008 and 2020 : 
Films :
